In [1]:
import redis
import json

In [2]:
REDIS = redis.Redis(host='data_store')

In [10]:
#Test
REDIS.set('my_incrementer', 42)

True

In [3]:
#Manager needs to do many things. One of which is manage the the company list and populate the data-store
companies = {
    "AAPL":"Apple",
    "FB":"Facebook",
    "GOOG":"Google"
    }
REDIS.set('companies', json.dumps(companies))

True

In [5]:
REDIS.get('IEX_Stock_Count')

In [ ]:
REDIS.get('Twitter_Stock_Count')

In [6]:
#TEst
REDIS.set('test',1)

True

In [11]:
int(REDIS.get('test')) ==1

True

In [12]:
#When setting up the flag, need to setup scheudle for the time, but also need to check if after open time
# and before close time, and set the flag to true.

In [19]:
!pip install tzlocal

  Running setup.py bdist_wheel for tzlocal ... done
  Stored in directory: /home/ds/.cache/pip/wheels/7c/a1/5d/0f37ce6eb6eea391bd185f5747429a93519be115d007263bcf
Successfully built tzlocal


### Manager

In [4]:
#Imports
import redis
import json
import schedule
import time
import datetime
#from tzlocal import get_localzone

import mysql.connector
from sqlalchemy import create_engine
from sqlalchemy import inspect
from sqlalchemy import MetaData
from sqlalchemy import Table
from sqlalchemy import Column
from sqlalchemy import Integer, String, DateTime

#Connect to the DataStore
REDIS = redis.Redis(host='data_store')

In [4]:
#Check environment variables
%env DB_USER

'stream'

In [5]:
#Setup Database
User = %env DB_USER
password = %env DB_PWD
dbname = %env DB_NAME


engine = create_engine('mysql+mysqlconnector://{}:{}@db:3306/{}'.format(User, password, dbname), echo=False)
inspector = inspect(engine)



#Create table object
meta = MetaData(engine)
if not engine.dialect.has_table(engine, 'log'):
    print('Log Table does not exist')
    print('Log Table being created....')
    #Time, Source, Current Count, Count Diff
    t1 = Table('log', meta,
                 Column('log_time', DateTime, default=datetime.datetime.utcnow),
                 Column('Source', String(30)),
                 Column('Current_Count', Integer),
                 Column('Count_Diff', Integer))
    t1.create()
else:
    print('Table Exists')
    

Log Table does not exist
Log Table being created....


In [9]:
############ Test insert ##################
conn = engine.connect()
ins = t1.insert().values(
            log_time = datetime.datetime.now(),
            Source = 'Test',
            Current_Count=42,
            Count_Diff=5
               )
conn.execute(ins)

In [6]:
#Set up the company list
companies = {
    "AAPL":"Apple",
    "FB":"Facebook",
    "GOOG":"Google"
    }
REDIS.set('companies', json.dumps(companies))

True

In [ ]:
#Setup the Data ON Flag and default to 0
REDIS.set('Data_On',0)

#Create Explicit Data On/Off Functions
def DataOn():
    REDIS.set('Data_On',1)

def DataOff():
    REDIS.set('Data_On',0)
    
global past_twitter_count
past_tweet_count = 0


###################### Will Put this in each of the sources ################################
    
def log():
    #Need to log: Time, Source, Current Count, Count Diff
    now = datetime.datetime.now()
    current_twitter_count = int(REDIS.get('Twitter_Stock_Count'))
    current_stock_count = int(REDIS.get('IEX_Stock_Count'))
    
    global past_twitter_count
    twitter_count_diff = current_twitter_count - past_twitter_count
    past_twitter_count = current_twitter_count
    
    global past_stock_count
    stock_count_diff = current_stock_count - past_stock_count
    past_stock_count = current_stock_count
    
    #Need to log these items to a database.
    ins = table.insert().values(
            datetime = now,
            current_twitter_count=current_twitter_count,
            current_stock_count=current_stock_count,
            twitter_count_dff=twitter_count_diff
            twitter_count_
        
            )
    

In [15]:
#Turn on if starts during open times
now = datetime.datetime.now()

if now.hour >= 14 and now.hour < 21:
    if now.hour = 14 and now.minute >= 30:
        DataOn()
    elif now.hour > 14:
        DataOn()

17

In [ ]:
#Setup Schedules. Time in 24 hour clocks
schedule.clear()
#Setup Data On/Off Schedule
schedule.every().monday.at("14:30").do(DataOn)
schedule.every().monday.at("21:00").do(DataOff)
schedule.every().tuesday.at("14:30").do(DataOn)
schedule.every().tuesday.at("21:00").do(DataOff)
schedule.every().wednesday.at("14:30").do(DataOn)
schedule.every().wednesday.at("21:00").do(DataOff)
schedule.every().thursday.at("14:30").do(DataOn)
schedule.every().thursday.at("21:00").do(DataOff)
schedule.every().friday.at("14:30").do(DataOn)
schedule.every().friday.at("21:00").do(DataOff)
